In [ ]:
# %% [markdown]
# # Análise de Impacto de Inundação em Edificações (Fluxo Interativo)
#
# Este notebook Jupyter guia através das etapas para analisar o impacto potencial de uma mancha de inundação simulada sobre edificações obtidas do OpenStreetMap (OSM). O fluxo é dividido em células para permitir a inspeção de resultados intermediários e a interação do usuário (como a inserção de coordenadas do exutório).
#
# **Bibliotecas Globais:** Importa bibliotecas usadas em múltiplas células.

# %%
# --- CÉLULA 0: Configuração e Bibliotecas Globais ---
import rasterio
import rasterio.features
import geopandas as gpd

import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
from shapely.geometry import box, shape, LineString
from pyproj import CRS
import warnings
import os

try:
    from pysheds.grid import Grid
except ImportError:
    print("Erro: Biblioteca 'pysheds' não encontrada. Instale com 'pip install pysheds'")
    # Em um notebook, talvez não queira sair, mas alertar.
    # exit()
    raise

# Ignorar warnings futuros
warnings.filterwarnings("ignore", category=FutureWarning)
print("Bibliotecas globais importadas com sucesso.")

In [ ]:
# --- Parâmetros Globais ---
# Ajuste estes caminhos e valores conforme necessário
INPUT_MDE_PATH = 'ponto_exutorio_auxiliar/srtm_data.tif'
OUTPUT_DIR = 'resultados_inundacao' # Diretório para salvar os resultados

# Parâmetros hidrológicos (ajuste conforme seu MDE e área)
STREAM_THRESHOLD = 1000 # Limiar de acumulação para definir rios (Célula 2)
CHANNEL_DEPTH = 10.0 # Profundidade do canal em metros (Célula 3)

# Cria o diretório de saída se não existir
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Diretório de saída: '{OUTPUT_DIR}'")


In [3]:
# %% [markdown]
# ---
# ## CÉLULA 1: Cálculo de Direção e Acumulação de Fluxo
#
# **Objetivo:** Carregar o MDE, realizar o condicionamento hidrológico inicial e calcular os rasters de direção de fluxo (`fdir`) e acumulação de fluxo (`acc`). Estes mapas são úteis para visualização inicial e para identificar possíveis pontos de exutório.
# **Bibliotecas Principais:** `pysheds`, `rasterio`, `numpy`.
# **Entrada:**
#   - Arquivo MDE (`INPUT_MDE_PATH`).
# **Saída:**
#   - Raster de Direção de Fluxo (`flow_direction.tif`).
#   - Raster de Acumulação de Fluxo (`flow_accumulation.tif`).
#   - Variáveis em memória para a próxima célula: `grid`, `inflated_dem`, `fdir`, `acc`.

# %%
# --- CÉLULA 1: Código ---
print("\n--- Iniciando Célula 1: Direção e Acumulação de Fluxo ---")

try:
    if not os.path.exists(INPUT_MDE_PATH):
        raise FileNotFoundError(f"Arquivo MDE '{INPUT_MDE_PATH}' não encontrado.")

    print("Carregando MDE e instanciando Grid PySheds...")
    grid = Grid.from_raster(INPUT_MDE_PATH, data_name='dem')
    dem = grid.read_raster(INPUT_MDE_PATH)

    # Condicionamento hidrológico
    print("Condicionando MDE (fill_pits, fill_depressions, resolve_flats)...")
    pit_filled_dem = grid.fill_pits(dem)
    flooded_dem = grid.fill_depressions(pit_filled_dem)
    inflated_dem = grid.resolve_flats(flooded_dem) # MDE condicionado

    # Direção e acumulação do fluxo
    print("Calculando direção e acumulação do fluxo...")
    dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
    fdir = grid.flowdir(inflated_dem, dirmap=dirmap)
    acc = grid.accumulation(fdir, dirmap=dirmap)

    # Salvar rasters intermediários
    fdir_path = os.path.join(OUTPUT_DIR, 'flow_direction.tif')
    acc_path = os.path.join(OUTPUT_DIR, 'flow_accumulation.tif')

    print(f"Salvando raster de direção de fluxo em '{fdir_path}'...")
    grid.to_raster(fdir, fdir_path, blockxsize=16, blockysize=16, nodata=-9999) # Exemplo com nodata e blocos

    print(f"Salvando raster de acumulação de fluxo em '{acc_path}'...")
    grid.to_raster(acc, acc_path, blockxsize=16, blockysize=16, nodata=-9999) # Exemplo com nodata e blocos

    print("Célula 1 concluída com sucesso!")
    print("Variáveis `grid`, `inflated_dem`, `fdir`, `acc` disponíveis para a próxima célula.")
    print(f"Arquivos gerados: '{fdir_path}', '{acc_path}'")
    print(">>> Use um SIG (ex: QGIS) para visualizar os rasters e obter as coordenadas do exutório <<<")


except FileNotFoundError as e:
     print(f"Erro na Célula 1: {e}. Verifique o caminho do MDE.")
     # Pode ser útil levantar o erro para parar a execução do notebook
     raise
except Exception as e:
    print(f"Erro inesperado na Célula 1: {e}")
    raise


Calculando direção e acumulação do fluxo...
Salvando raster de direção de fluxo em 'resultados_inundacao\flow_direction.tif'...
Salvando raster de acumulação de fluxo em 'resultados_inundacao\flow_accumulation.tif'...
Célula 1 concluída com sucesso!
Variáveis `grid`, `inflated_dem`, `fdir`, `acc` disponíveis para a próxima célula.
Arquivos gerados: 'resultados_inundacao\flow_direction.tif', 'resultados_inundacao\flow_accumulation.tif'
>>> Use um SIG (ex: QGIS) para visualizar os rasters e obter as coordenadas do exutório <<<


In [4]:
# %%
# --- CÉLULA 2: Código Atualizado com Lógica Strahler Otimizada ---
print("\n--- Iniciando Célula 2: Delimitação da Bacia e Canais ---")

# --- !!! DEFINIR COORDENADAS DO EXUTÓRIO AQUI !!! ---
# Substitua pelos valores obtidos visualizando os mapas da Célula 1
outlet_coords = (-44.118627, -20.316243) # (Longitude, Latitude) - EXEMPLO
# -----------------------------------------------------

# Verificar se as variáveis da Célula 1 existem
if 'grid' not in locals() or 'fdir' not in locals() or 'acc' not in locals() or 'inflated_dem' not in locals():
    print("Erro: Variáveis da Célula 1 não encontradas. Execute a Célula 1 primeiro.")
    raise NameError("Variáveis da Célula 1 ausentes.")

# Importações necessárias para a lógica do Strahler
import numpy as np # Importante para np.isnan
from shapely.geometry import LineString, shape # Para vetorização
import geopandas as gpd # Para GeoDataFrame
import os # Para path
import rasterio.features # Para vetorização da bacia

try:
    print(f"Usando exutório em: {outlet_coords}")
    x_outlet, y_outlet = outlet_coords

    # Máscara de rios (baseada no limiar global)
    streams_mask = acc > STREAM_THRESHOLD

    # Snap do exutório à rede de drenagem
    print("Realizando snap do exutório...")
    try:
        x_snap, y_snap = grid.snap_to_mask(streams_mask, (x_outlet, y_outlet))
    except ValueError as e:
        print(f"Erro ao fazer snap: {e}. Verifique se as coordenadas estão dentro da área do MDE e próximas a um canal com acumulação > {STREAM_THRESHOLD}.")
        raise

    # Delimitação da Bacia
    print("Delimitação bacia hidrográfica...")
    catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, dirmap=dirmap, xytype='coordinate')

    # Recorte do grid para a bacia (modifica o objeto 'grid' em memória)
    print("Recortando grid para a bacia...")
    grid.clip_to(catch)
    print("Grid recortado. As próximas operações serão limitadas à bacia.")

    # ---------------------------------------------------------------------
    # CÁLCULO DISTÂNCIA DE FLUXO E EXPORTAÇÃO
    # ---------------------------------------------------------------------
    # print("Calculando a distância de fluxo para o exutório (Flow Distance)...")
    # dist = grid.distance_to_outlet(x=x_snap, y=y_snap, fdir=grid.view(fdir), dirmap=dirmap,
    #                                xytype='coordinate')
    #
    # dist_path = os.path.join(OUTPUT_DIR, 'flu_distance.tif')
    # grid.to_raster(dist, dist_path, dtype=rasterio.float32)
    # print(f"Distância de Fluxo salva em: '{dist_path}'")
    # ---------------------------------------------------------------------

    # ---------------------------------------------------------------------
    # EXTRAÇÃO DA REDE E CÁLCULO STRAHLER (LÓGICA OTIMIZADA)
    # ---------------------------------------------------------------------
    print("Extraindo rede de drenagem e calculando ordem de Strahler (Raster)...")

    # 1. Obter fdir e streams_mask recortadas
    fdir_clipped = grid.view(fdir) # fdir recortado
    # Requer o recalculo da ACC para garantir que a máscara corresponda ao recorte
    clipped_acc = grid.accumulation(fdir_clipped, dirmap=dirmap)
    clipped_streams_mask = clipped_acc > STREAM_THRESHOLD

    # 2. CÁLCULO DO RASTER DE ORDEM DE STRAHLER
    stream_order_raster = grid.stream_order(fdir_clipped, clipped_streams_mask)

    # 3. EXTRAÇÃO DA REDE VETORIAL
    print("Extraindo a rede de drenagem como feições vetoriais...")
    network = grid.extract_river_network(fdir_clipped, clipped_streams_mask, distance=1)

    # Vetorizar Bacia (MANTIDO O MESMO CÓDIGO ANTERIOR)
    print("Vetorizando a bacia...")
    catch_view = grid.view(catch)
    catchment_shapes = rasterio.features.shapes(
        catch_view.astype(np.uint8),
        mask=catch_view,
        transform=grid.viewfinder.affine
    )
    catchment_geoms = [shape(geom) for geom, value in catchment_shapes if value == 1]
    if not catchment_geoms:
        raise RuntimeError("Não foi possível vetorizar a bacia.")

    catchment_gdf_temp = gpd.GeoDataFrame(geometry=catchment_geoms, crs=grid.viewfinder.crs)
    catchment_gdf_temp['id'] = 1
    catchment_gdf = catchment_gdf_temp.dissolve(by='id')

    bacia_path = os.path.join(OUTPUT_DIR, 'bacia.geojson')
    catchment_gdf.to_file(bacia_path, driver='GeoJSON')
    print(f"Bacia salva em: '{bacia_path}'")


    # 4. ATRIBUIÇÃO DO ATRIBUTO (Raster -> Vetor)
    print("Atribuindo ordem de Strahler aos canais vetorizados...")
    features = network.get('features', [])
    geometries = []
    strahler_orders = []

    if not features:
        print("Aviso: Nenhuma feição de canal encontrada na rede extraída.")
        canais_path = None # Indica que não há canais para salvar
    else:
        # Dados do Viewfinder do grid recortado para mapeamento
        vf_affine = grid.viewfinder.affine
        inv_vf_affine = ~vf_affine
        vf_height, vf_width = grid.viewfinder.shape

        for feature in features:
            coords = feature['geometry']['coordinates']
            # Cria LineString apenas se tiver 2 ou mais pontos
            if len(coords) >= 2:
                line = LineString(coords)
                geometries.append(line)

                # Mapeamento do ponto inicial da linha para a célula do raster
                start_coord = coords[0]
                col_frac, row_frac = inv_vf_affine * (start_coord[0], start_coord[1])
                row, col = int(row_frac), int(col_frac)

                order_val = None
                # Verificação de segurança para os índices do array
                if 0 <= row < vf_height and 0 <= col < vf_width:
                    # Lendo o valor de Strahler do raster
                    order_val = stream_order_raster[row, col]

                # Convertendo para inteiro (se não for nodata/None)
                if order_val is not None and not np.isnan(order_val):
                    strahler_orders.append(int(order_val))
                else:
                    strahler_orders.append(None)
            else:
                 print(f"Aviso: Segmento de canal com menos de 2 pontos encontrado e ignorado: {coords}.")


        # 5. CRIAR E SALVAR O GEODATAFRAME COM O ATRIBUTO
        if geometries:
            # Reintroduzindo a coluna 'strahler_order'
            streams_gdf = gpd.GeoDataFrame({'strahler_order': strahler_orders}, geometry=geometries, crs=grid.viewfinder.crs)
            # Nome de arquivo ligeiramente alterado para refletir a otimização
            canais_path = os.path.join(OUTPUT_DIR, 'canais_strahler.geojson')
            streams_gdf.to_file(canais_path, driver='GeoJSON')
            print(f"Canais (com ordem de Strahler - Lógica Otimizada) salvos em: '{canais_path}'")
        else:
            print("Nenhum canal válido para salvar.")
            canais_path = None


    print("Célula 2 concluída com sucesso!")
    print("Variáveis `grid` (recortado), `catch` (máscara) disponíveis.")
    print(f"Arquivos gerados: '{bacia_path}', '{canais_path if canais_path else 'Nenhum canal salvo'}'") #, '{dist_path}'")

except Exception as e:
    print(f"Erro inesperado na Célula 2: {e}")
    raise


--- Iniciando Célula 2: Delimitação da Bacia e Canais ---
Usando exutório em: (-44.118627, -20.316243)
Realizando snap do exutório...
Delimitação bacia hidrográfica...
Recortando grid para a bacia...
Grid recortado. As próximas operações serão limitadas à bacia.
Extraindo rede de drenagem e calculando ordem de Strahler (Raster)...
Extraindo a rede de drenagem como feições vetoriais...
Vetorizando a bacia...
Bacia salva em: 'resultados_inundacao\bacia.geojson'
Atribuindo ordem de Strahler aos canais vetorizados...
Canais (com ordem de Strahler - Lógica Otimizada) salvos em: 'resultados_inundacao\canais_strahler.geojson'
Célula 2 concluída com sucesso!
Variáveis `grid` (recortado), `catch` (máscara) disponíveis.
Arquivos gerados: 'resultados_inundacao\bacia.geojson', 'resultados_inundacao\canais_strahler.geojson'


## DISTANCIA DO FLUXO

In [ ]:
# REMOVER/COMENTAR ESTAS LINHAS DA CÉLULA 2:
# ---------------------------------------------------------------------
# CÁLCULO DISTÂNCIA DE FLUXO E EXPORTAÇÃO
# ---------------------------------------------------------------------
print("Calculando a distância de fluxo para o exutório (Flow Distance)...")
dist = grid.distance_to_outlet(x=x_snap, y=y_snap, fdir=grid.view(fdir), dirmap=dirmap,
                               xytype='coordinate')

dist_path = os.path.join(OUTPUT_DIR, 'flu_distance.tif')
grid.to_raster(dist, dist_path, dtype=rasterio.float32)
print(f"Distância de Fluxo salva em: '{dist_path}'")
# ---------------------------------------------------------------------

In [ ]:
# # %%
# # --- CÉLULA 3: Cálculo Isolado da Distância de Fluxo ---
# print("\n--- Iniciando Célula 3: Cálculo da Distância de Fluxo (Flow Distance) ---")
#
# # Verificar se as variáveis essenciais da Célula 2 existem
# if 'grid' not in locals() or 'fdir' not in locals() or 'x_snap' not in locals() or 'y_snap' not in locals():
#     print("Erro: Variáveis da Célula 2 ('grid', 'fdir', 'x_snap', 'y_snap') não encontradas. Execute a Célula 2 primeiro.")
#     raise NameError("Variáveis da Célula 2 ausentes ou não definidas.")
#
# # O objeto 'grid' já está recortado para a bacia (graças ao grid.clip_to(catch) na Célula 2)
# # 'x_snap' e 'y_snap' são as coordenadas do exutório ajustadas (snap)
#
# try:
#     # Importações necessárias
#     import os
#     import rasterio.float32
#
#     # A Célula 2 usou grid.view(fdir) para garantir que fdir estivesse recortado.
#     # Usaremos fdir_clipped que já foi definida na Célula 2, mas podemos usar grid.view(fdir)
#     # ou grid.view(fdir_clipped) novamente para garantir o uso da versão recortada.
#     fdir_clipped_view = grid.view(fdir)
#
#     # ---------------------------------------------------------------------
#     # CÁLCULO DISTÂNCIA DE FLUXO
#     # ---------------------------------------------------------------------
#     print(f"Calculando a distância de fluxo para o exutório ({x_snap:.6f}, {y_snap:.6f})...")
#
#     # O cálculo em si (seguindo a estrutura do manual: dist = grid.distance_to_outlet(x, y, fdir=fdir, xytype='coordinate'))
#     dist = grid.distance_to_outlet(x=x_snap,
#                                    y=y_snap,
#                                    fdir=fdir_clipped_view, # Usa a fdir recortada para a bacia
#                                    dirmap=dirmap, # dirmap é necessário
#                                    xytype='coordinate')
#
#     # ---------------------------------------------------------------------
#     # EXPORTAÇÃO (Salvando o resultado)
#     # ---------------------------------------------------------------------
#     dist_path = os.path.join(OUTPUT_DIR, 'flu_distance_celula3.tif') # Nome de arquivo ligeiramente alterado para evitar conflito
#     grid.to_raster(dist, dist_path, dtype=rasterio.float32)
#     print(f"Distância de Fluxo salva em: '{dist_path}'")
#     # ---------------------------------------------------------------------
#
#     print("Célula 3 concluída com sucesso! Variável 'dist' disponível.")
#
# except Exception as e:
#     print(f"Erro inesperado na Célula 3: {e}")
#     raise

In [ ]:
# %% [markdown]
# ---
# ## CÉLULA 3: Cálculo do HAND, Geração do Raster e Vetor de Inundação
#
# **Objetivo:** Calcular o raster HAND (Height Above Nearest Drainage) dentro da bacia delimitada, gerar o raster final da mancha de inundação com base na profundidade de canal definida, e também vetorizar essa mancha de inundação.
# **Bibliotecas Principais:** `pysheds`, `rasterio`, `numpy`, `geopandas`, `shapely`.
# **Entrada:**
#   - Variáveis das células anteriores: `grid` (recortado), `fdir` (original), `inflated_dem` (original), `acc` (original).
#   - Máscara da Bacia (`catch`).
#   - Profundidade do Canal (`CHANNEL_DEPTH`) - definida na Célula 0.
#   - Limiar de Acumulação (`STREAM_THRESHOLD`) - definido na Célula 0.
# **Saída:**
#   - Raster da Mancha de Inundação (`inundacao_mapa.tif`).
#   - Vetor (Polígono/Multipolígono) da Mancha de Inundação (`inundacao.geojson`).
#   - Variáveis em memória: `inundacao_raster_path`, `inundacao_vetor_path`.

# %%
# --- CÉLULA 3: Código ---
print("\n--- Iniciando Célula 3: HAND, Raster e Vetor de Inundação ---")

# Verificar se as variáveis necessárias existem
if 'grid' not in locals() or 'fdir' not in locals() or 'inflated_dem' not in locals() or 'catch' not in locals() or 'acc' not in locals():
    print("Erro: Variáveis das Células 1 ou 2 não encontradas. Execute as células anteriores.")
    raise NameError("Variáveis das Células 1 ou 2 ausentes.")

try:
    # Máscara de rios (baseada no limiar global, usada pelo compute_hand)
    streams_mask_global = acc > STREAM_THRESHOLD

    # HAND (calculado dentro da bacia usando a visão do grid)
    print("Calculando HAND dentro da bacia...")
    hand = grid.compute_hand(fdir, inflated_dem, streams_mask_global)

    # Recorte do HAND para a bacia (aplicando a máscara novamente via grid.view)
    print(f"Calculando mancha de inundação (profundidade) para {CHANNEL_DEPTH}m...")
    hand_view = grid.view(hand, nodata=np.nan) # HAND dentro da bacia
    # Calcula a profundidade da inundação onde HAND < channel_depth
    inundation_depth = np.where(hand_view < CHANNEL_DEPTH, CHANNEL_DEPTH - hand_view, np.nan) # Usar NaN como nodata

    # --- Salvar Raster de Inundação (necessário para Célula 4) ---
    print("Construindo perfil do raster de saída...")
    profile = {
        'crs': grid.viewfinder.crs,
        'transform': grid.viewfinder.affine,
        'height': grid.viewfinder.shape[0],
        'width': grid.viewfinder.shape[1],
        'driver': 'GTiff',
        'count': 1,
        'dtype': rasterio.float32,
        'nodata': np.nan
    }
    inundacao_raster_path = os.path.join(OUTPUT_DIR, 'inundacao_mapa.tif')
    with rasterio.open(inundacao_raster_path, 'w', **profile) as dst:
        dst.write(inundation_depth.astype(rasterio.float32), 1)
    print(f"Arquivo raster de inundação salvo como: '{inundacao_raster_path}'")


    # --- Vetorizar a Mancha de Inundação ---
    print("Vetorizando a mancha de inundação (área)...")
    # Criar uma máscara booleana onde a inundação existe (profundidade não é NaN)
    inundation_mask = ~np.isnan(inundation_depth)

    if not np.any(inundation_mask):
         print("Aviso: Nenhuma área inundada encontrada para vetorizar.")
         inundacao_vetor_path = None
    else:
        # Vetorizar usando a máscara (value será 1 onde a máscara for True)
        # Usamos a mesma transformação do raster de profundidade
        shapes_generator = rasterio.features.shapes(
            inundation_mask.astype(np.uint8), # Precisa ser tipo inteiro
            mask=inundation_mask, # Vetoriza apenas onde a máscara é True
            transform=profile['transform'] # Usa a transformação do raster salvo
        )

        # Coletar geometrias onde o valor do pixel vetorizado é 1 (True)
        flood_geometries = [shape(geom) for geom, value in shapes_generator if value == 1]

        if not flood_geometries:
            print("Aviso: Nenhuma geometria encontrada após a vetorização da máscara de inundação.")
            inundacao_vetor_path = None
        else:
            # Criar GeoDataFrame temporário
            flood_gdf_temp = gpd.GeoDataFrame(geometry=flood_geometries, crs=profile['crs'])
            # Adicionar ID para dissolver
            flood_gdf_temp['id'] = 1
            # Dissolver para obter um único MultiPolygon (ou Polygon)
            flood_gdf_dissolved = flood_gdf_temp.dissolve(by='id')

            # Salvar o vetor GeoJSON
            inundacao_vetor_path = os.path.join(OUTPUT_DIR, 'inundacao.geojson')
            flood_gdf_dissolved.to_file(inundacao_vetor_path, driver='GeoJSON')
            print(f"Arquivo vetorial de inundação salvo como: '{inundacao_vetor_path}'")

    print(f"Célula 3 concluída com sucesso!")
    print(f"Variáveis `inundacao_raster_path` e `inundacao_vetor_path` disponíveis.")

except Exception as e:
    print(f"Erro inesperado na Célula 3: {e}")
    raise

## DOWNLOAD OSMNX ( Podemos substituir essa etapa por qualquer vetor de escolha)

In [ ]:
# %% [markdown]
# ---
# ## CÉLULA 4: Download de Dados OSM (Edificações, Vias, Comodidades)
#
# **Objetivo:** Usar a extensão do raster de inundação para definir uma AOI, baixar dados de múltiplas categorias (edificações, vias, comodidades) do OpenStreetMap usando OSMnx e salvá-los (inicialmente filtrando polígonos).
# **Bibliotecas Principais:** `osmnx`, `geopandas`, `rasterio`, `shapely`, `pyproj`.
# **Entrada:**
#   - Caminho do Raster de Inundação (`inundacao_raster_path`) - da Célula 3.
# **Saída:**
#   - Arquivo GeoPackage com **Polígonos** OSM (`osm_polygons.gpkg`). Outros tipos (linhas, pontos) são baixados mas filtrados antes de salvar neste exemplo.
#   - Variável em memória: `osm_polygons_path`.

# %%
# --- CÉLULA 4: Código Modificado ---
print("\n--- Iniciando Célula 4: Download de Dados OSM (Múltiplas Tags) ---")

# Verificar se a variável da Célula 3 existe
if 'inundacao_raster_path' not in locals() or not os.path.exists(inundacao_raster_path):
    print("Erro: Caminho do raster de inundação não encontrado. Execute a Célula 3 primeiro.")
    raise NameError("Variável `inundacao_raster_path` ausente ou arquivo inválido.")

# Define o nome do arquivo de saída para polígonos OSM
osm_polygons_path = os.path.join(OUTPUT_DIR, 'osm_polygons.gpkg') # Nome mais genérico

try:
    # --- Definir AOI a partir do Raster de Inundação ---
    print(f"Lendo raster '{inundacao_raster_path}' para definir AOI...")
    with rasterio.open(inundacao_raster_path) as src:
        flood_crs = src.crs
        flood_bounds = src.bounds

    aoi_bbox_native = box(*flood_bounds)
    aoi_gdf_native = gpd.GeoDataFrame([1], geometry=[aoi_bbox_native], crs=flood_crs)

    # Reprojetar para EPSG:4326 (necessário para OSMnx)
    target_crs_epsg = 4326
    if flood_crs.to_epsg() != target_crs_epsg:
        print(f"Reprojetando AOI de {flood_crs.to_string()} para EPSG:{target_crs_epsg}...")
        aoi_gdf_4326 = aoi_gdf_native.to_crs(epsg=target_crs_epsg)
    else:
        print("AOI já está em EPSG:4326.")
        aoi_gdf_4326 = aoi_gdf_native

    aoi_polygon_4326 = aoi_gdf_4326.geometry.iloc[0]
    print(f"AOI definida e em EPSG:{target_crs_epsg}.")

    # --- Coletar Dados OSM (Múltiplas Tags) ---
    print("Baixando dados do OpenStreetMap (Edificações, Vias, Comodidades, Uso do Solo)...")
    # Define as tags a serem buscadas (chave: True busca qualquer valor para a chave)
    tags_to_fetch = {
        'building': True,
        'highway': True,
        'amenity': True,
        'landuse': ['residential', 'commercial', 'industrial', 'retail'], # Exemplo específico
        'natural': ['water'], # Exemplo
        'leisure': ['park', 'playground'] # Exemplo
        # Adicione outras tags conforme necessário
    }
    osm_features_gdf = ox.features_from_polygon(aoi_polygon_4326, tags_to_fetch)
    print(f"Download concluído. {len(osm_features_gdf)} feições encontradas inicialmente (tipos mistos).")

    # --- Filtrar **APENAS POLÍGONOS** para salvar neste arquivo ---
    # Nota: As feições de linha (highway) e ponto (algumas amenities) serão descartadas aqui.
    # Você pode querer salvar diferentes tipos em arquivos separados depois.
    osm_polygons_gdf = osm_features_gdf[
        osm_features_gdf.geometry.notna() &
        osm_features_gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])
    ].copy() # Usar .copy() para evitar SettingWithCopyWarning
    print(f"Filtrado para {len(osm_polygons_gdf)} feições do tipo Polígono/Multipolígono válidas.")

    # Salvar os Polígonos
    if osm_polygons_gdf.empty:
        print("Aviso: Nenhuma feição poligonal encontrada na AOI definida para as tags selecionadas.")
        print("Célula 4 concluída. Nenhum arquivo de polígonos salvo.")
        osm_polygons_path = None # Indica que não há polígonos
    else:
        # Garante o CRS antes de salvar
        if osm_polygons_gdf.crs is None:
             osm_polygons_gdf.crs = CRS.from_epsg(target_crs_epsg)
        osm_polygons_gdf.to_file(osm_polygons_path, driver="GPKG")
        print(f"Célula 4 concluída com sucesso!")
        print(f"Feições poligonais salvas em: '{osm_polygons_path}'")

    print(f"Variável `osm_polygons_path` disponível (pode ser None).")
    # Lembre-se que osm_features_gdf contém todos os tipos baixados (pontos, linhas, polígonos)
    # Você pode usá-lo para salvar outros tipos, se desejar:
    # osm_lines_gdf = osm_features_gdf[osm_features_gdf.geometry.type.isin(['LineString', 'MultiLineString'])].copy()
    # if not osm_lines_gdf.empty: osm_lines_gdf.to_file(os.path.join(OUTPUT_DIR, 'osm_lines.gpkg'), driver="GPKG")
    # osm_points_gdf = osm_features_gdf[osm_features_gdf.geometry.type == 'Point'].copy()
    # if not osm_points_gdf.empty: osm_points_gdf.to_file(os.path.join(OUTPUT_DIR, 'osm_points.gpkg'), driver="GPKG")


except Exception as e:
    print(f"Erro inesperado na Célula 4: {e}")
    raise

## ESTATÍSTICAS

In [ ]:
# %% [markdown]
# ---
# ## CÉLULA 5: Análise de Interseção (Inundação vs Polígonos OSM) e Estatísticas
#
# **Objetivo:** Carregar a mancha de inundação vetorizada (`.geojson`) e os polígonos OSM (`.gpkg` da Célula 4), **filtrar polígonos indesejados (ex: 'water')**, realizar a interseção espacial para identificar polígonos OSM afetados e calcular estatísticas básicas.
# **Bibliotecas Principais:** `geopandas`, `pyproj`.
# **Entrada:**
#   - Caminho do Vetor de Inundação (`inundacao_vetor_path`) - da Célula 3 (pode ser `None`).
#   - Caminho do GeoPackage de Polígonos OSM (`osm_polygons_path`) - da Célula 4 (pode ser `None`).
# **Saída:**
#   - Impressão das estatísticas (contagem de polígonos OSM exclui 'water').
#   - Variáveis em memória: `flood_analysis_gdf`, `osm_polygons_gdf` (filtrado), `affected_polygons_gdf`, `count_affected_polygons`, `total_polygons_aoi`.

# %%
# --- CÉLULA 5: Código (Adaptado para osm_polygons e com filtro de 'water') ---
print("\n--- Iniciando Célula 5: Análise de Interseção (Inundação vs Polígonos OSM) e Estatísticas ---")

# Verificar se as variáveis necessárias das células anteriores existem
if 'inundacao_vetor_path' not in locals():
     print("Erro: Variável `inundacao_vetor_path` não definida. Execute a Célula 3.")
     raise NameError("Variável `inundacao_vetor_path` ausente.")
if 'osm_polygons_path' not in locals():
     print("Erro: Variável `osm_polygons_path` não definida. Execute a Célula 4.")
     raise NameError("Variável `osm_polygons_path` ausente.")

try:
    # --- Carregar Vetor de Inundação ---
    if inundacao_vetor_path is None or not os.path.exists(inundacao_vetor_path):
        print("Aviso: Arquivo vetorial de inundação não encontrado ou não gerado na Célula 3.")
        flood_crs_fallback = None
        try:
             with rasterio.open(inundacao_raster_path) as src_fallback: # Usa o path do raster da Célula 3
                 flood_crs_fallback = src_fallback.crs
        except Exception:
             print("Aviso: Não foi possível obter CRS do raster de fallback.")
        flood_analysis_gdf = gpd.GeoDataFrame(geometry=[], crs=flood_crs_fallback)
    else:
        print(f"Carregando vetor de inundação '{inundacao_vetor_path}'...")
        flood_analysis_gdf = gpd.read_file(inundacao_vetor_path)
        if len(flood_analysis_gdf) > 1:
             print("Aviso: Vetor de inundação contém múltiplas features. Dissolvendo novamente...")
             flood_analysis_gdf['temp_id'] = 1
             flood_analysis_gdf = flood_analysis_gdf.dissolve(by='temp_id')
        print(f"Vetor de inundação carregado (CRS: {flood_analysis_gdf.crs}).")


    # --- Carregar Polígonos OSM e Harmonizar CRS ---
    osm_polygons_input_path = osm_polygons_path
    if osm_polygons_input_path is None or not os.path.exists(osm_polygons_input_path):
        print("Arquivo de polígonos OSM não encontrado. Análise de interseção não será realizada.")
        osm_polygons_gdf_input = gpd.GeoDataFrame(geometry=[], crs=CRS.from_epsg(4326)) # EPSG:4326
    else:
        osm_polygons_gdf_input = gpd.read_file(osm_polygons_input_path)
        print(f"Vetor de polígonos OSM '{osm_polygons_input_path}' carregado (CRS: {osm_polygons_gdf_input.crs}).")

    # Determinar o CRS alvo
    target_crs = flood_analysis_gdf.crs if not flood_analysis_gdf.empty and flood_analysis_gdf.crs else osm_polygons_gdf_input.crs

    if osm_polygons_gdf_input.empty:
        print("GeoDataFrame de polígonos OSM vazio.")
        osm_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=target_crs)
    elif osm_polygons_gdf_input.crs != target_crs and target_crs is not None:
        print(f"Reprojetando polígonos OSM de {osm_polygons_gdf_input.crs} para {target_crs}...")
        osm_polygons_gdf = osm_polygons_gdf_input.to_crs(target_crs)
    elif target_crs is None:
         print("Aviso: Não foi possível determinar um CRS alvo para harmonização.")
         osm_polygons_gdf = osm_polygons_gdf_input
    else:
        print("CRS dos polígonos OSM e inundação são compatíveis.")
        osm_polygons_gdf = osm_polygons_gdf_input


    # <<< INÍCIO DA MODIFICAÇÃO: Filtrar polígonos de 'water' >>>
    print(f"Total de polígonos OSM antes da filtragem: {len(osm_polygons_gdf)}")

    # Condição 1: Excluir se a coluna 'natural' existir e for 'water'
    if 'natural' in osm_polygons_gdf.columns:
        print("Filtrando polígonos com 'natural' == 'water'...")
        # Mantém apenas as linhas onde 'natural' NÃO é 'water'
        # .fillna('') é uma segurança caso a coluna 'natural' tenha NaNs
        osm_polygons_gdf = osm_polygons_gdf[
            osm_polygons_gdf['natural'].fillna('') != 'water'
        ]

    # Condição 2: Excluir se a coluna 'water' existir e tiver qualquer valor (não nulo)
    # Isso remove polígonos com tags como 'water=river', 'water=lake', 'water=pond' etc.
    if 'water' in osm_polygons_gdf.columns:
        print("Filtrando polígonos com a tag 'water' preenchida...")
        # Mantém apenas as linhas onde a coluna 'water' é Nula (NaN)
        osm_polygons_gdf = osm_polygons_gdf[
            osm_polygons_gdf['water'].isna()
        ]

    print(f"Total de polígonos OSM após a filtragem (sem 'water'): {len(osm_polygons_gdf)}")
    # <<< FIM DA MODIFICAÇÃO >>>


    # --- Análise de Interseção ---
    print("Realizando interseção espacial (Inundação vs Polígonos OSM)...")
    if not osm_polygons_gdf.empty and not flood_analysis_gdf.empty:
        if osm_polygons_gdf.crs is None or flood_analysis_gdf.crs is None:
             print("Aviso: Um dos GeoDataFrames está sem CRS definido. Interseção não realizada.")
             affected_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=target_crs)
        elif osm_polygons_gdf.crs != flood_analysis_gdf.crs:
             print(f"Aviso: CRS não coincidem ({osm_polygons_gdf.crs} vs {flood_analysis_gdf.crs}). Interseção não realizada.")
             affected_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=target_crs)
        else:
            # Nota: O sjoin agora usa o osm_polygons_gdf JÁ FILTRADO (sem 'water')
            affected_sjoin = gpd.sjoin(osm_polygons_gdf, flood_analysis_gdf, how='inner', predicate='intersects')
            affected_polygons_gdf = affected_sjoin[~affected_sjoin.index.duplicated(keep='first')] # GDF dos polígonos OSM afetados
    else:
        print("Não há dados de polígonos OSM (pós-filtro) ou inundação para realizar a interseção.")
        affected_polygons_gdf = gpd.GeoDataFrame(geometry=[], crs=target_crs)


    # --- Estatísticas ---
    count_affected_polygons = len(affected_polygons_gdf)
    # <<< MODIFICAÇÃO >>>: Contagem total agora reflete o GDF filtrado
    total_polygons_aoi = len(osm_polygons_gdf)

    # --- Imprimir Estatísticas ---
    print("-" * 30)
    print("Estatísticas da Análise de Impacto (Polígonos OSM):")
    # <<< MODIFICAÇÃO >>>: Texto atualizado para clareza
    print(f"Número total de polígonos OSM na AOI (excluindo 'water'): {total_polygons_aoi}")
    print(f"Número de polígonos OSM afetados pela inundação: {count_affected_polygons}")
    if total_polygons_aoi > 0:
        percent_affected = (count_affected_polygons / total_polygons_aoi) * 100
        print(f"Percentual de polígonos OSM afetados: {percent_affected:.2f}%")
    else:
         print(f"Percentual de polígonos OSM afetados: N/A (nenhum polígono OSM na AOI)")
    print("-" * 30)

    print("Célula 5 concluída com sucesso!")
    print("Variáveis `flood_analysis_gdf`, `osm_polygons_gdf`, `affected_polygons_gdf`, `count_affected_polygons`, `total_polygons_aoi` disponíveis.")

except Exception as e:
    print(f"Erro inesperado na Célula 5: {e}")
    # Define variáveis como vazias/zero para a Célula 6 não falhar se for executada
    flood_analysis_gdf = gpd.GeoDataFrame(geometry=[])
    osm_polygons_gdf = gpd.GeoDataFrame(geometry=[])
    affected_polygons_gdf = gpd.GeoDataFrame(geometry=[])
    count_affected_polygons = 0
    total_polygons_aoi = 0
    raise

## (INTERSECT + BUFFER) - Personalizaçao - SOLOS

In [ ]:
# %% [markdown]
# ---
# ## CÉLULA 6: Plotagem do Mapa de Impacto (com Bacia e Canais)
#
# **Objetivo:** Gerar e salvar uma visualização cartográfica mostrando a bacia hidrográfica, a rede de drenagem, a mancha de inundação, os polígonos OSM totais e os polígonos OSM afetados, com anotações estatísticas e legenda personalizada, garantindo a consistência do CRS e a ordem de plotagem correta.
# **Bibliotecas Principais:** `matplotlib`, `geopandas`, `matplotlib.patches`, `matplotlib.lines`.
# **Entrada:**
#   - Variáveis da Célula 5: `flood_analysis_gdf`, `osm_polygons_gdf`, `affected_polygons_gdf`, `count_affected_polygons`, `total_polygons_aoi`.
#   - Caminhos dos arquivos da Célula 2: `bacia_path`, `canais_path` (podem ser `None`).
#   - CRS Projetado (`TARGET_PROJECTED_CRS`) - da Célula 3b.
# **Saída:**
#   - Arquivo de Imagem do Mapa (`mapa_impacto_inundacao_poligonos_osm.png`).
#   - Plot exibido no notebook.

# %%

TARGET_PROJECTED_CRS = "EPSG:31983"
# --- CÉLULA 6: Código (CRS Harmonizado e zorder/Ordem Ajustados) ---
print("\n--- Iniciando Célula 6: Plotagem do Mapa (com Bacia e Canais) ---")

# (Verificações de variáveis e CRS como antes...)
# Verificar se as variáveis da Célula 5 existem
if ('flood_analysis_gdf' not in locals() or
    'osm_polygons_gdf' not in locals() or
    'affected_polygons_gdf' not in locals() or
    'count_affected_polygons' not in locals() or
    'total_polygons_aoi' not in locals()):
    print("Erro: Variáveis da Célula 5 não encontradas. Execute a Célula 5 primeiro.")
    raise NameError("Variáveis da Célula 5 ausentes.")

# Verificar se os paths da Célula 2 existem
if 'bacia_path' not in locals() or 'canais_path' not in locals():
     print("Erro: Variáveis `bacia_path` ou `canais_path` não definidas. Execute a Célula 2.")
     raise NameError("Variáveis da Célula 2 ausentes.")

# Garantir acesso ao CRS Projetado
try:
    plot_crs_target = TARGET_PROJECTED_CRS
    print(f"CRS alvo para plotagem: {plot_crs_target}")
except NameError:
    print("Aviso: Variável TARGET_PROJECTED_CRS não encontrada. Redefinindo para EPSG:31983.")
    plot_crs_target = "EPSG:31983"


# Importar Patch e Line2D
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

output_map_path = os.path.join(OUTPUT_DIR, 'mapa_impacto_inundacao_poligonos_osm.png')

try:
    # --- Carregar e Harmonizar CRS de TODAS as camadas ---
    print("Carregando e harmonizando CRS das camadas para plotagem...")
    # (Código de carregamento e harmonização como antes...)
    gdf_flood_plot = flood_analysis_gdf
    if not gdf_flood_plot.empty and gdf_flood_plot.crs != plot_crs_target:
        gdf_flood_plot = gdf_flood_plot.to_crs(plot_crs_target)

    gdf_polygons_plot = osm_polygons_gdf
    if not gdf_polygons_plot.empty and gdf_polygons_plot.crs != plot_crs_target:
        gdf_polygons_plot = gdf_polygons_plot.to_crs(plot_crs_target)

    gdf_affected_plot = affected_polygons_gdf
    if not gdf_affected_plot.empty and gdf_affected_plot.crs != plot_crs_target:
        gdf_affected_plot = gdf_affected_plot.to_crs(plot_crs_target)

    gdf_catchment_plot = None
    if bacia_path and os.path.exists(bacia_path):
        try:
            gdf_catchment_plot = gpd.read_file(bacia_path)
            if gdf_catchment_plot.crs != plot_crs_target:
                 gdf_catchment_plot = gdf_catchment_plot.to_crs(plot_crs_target)
            print(f"Bacia carregada e harmonizada.")
        except Exception as e: print(f"Aviso: Não foi possível carregar/harmonizar o arquivo da bacia: {e}")

    gdf_streams_plot = None
    if canais_path and os.path.exists(canais_path):
         try:
            gdf_streams_plot = gpd.read_file(canais_path)
            if gdf_streams_plot.crs != plot_crs_target:
                 gdf_streams_plot = gdf_streams_plot.to_crs(plot_crs_target)
            print(f"Canais carregados e harmonizados.")
         except Exception as e: print(f"Aviso: Não foi possível carregar/harmonizar o arquivo dos canais: {e}")


    # --- Plotagem ---
    print("Gerando mapa de visualização...")
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.set_aspect('equal')

    legend_handles = []
    legend_labels = []

    # --- Plotagem e Coleta de Handles para Legenda (Ordem e Linewidth Ajustados) ---

    # 1. Inundação (Fundo)
    color_inundacao = 'cyan'; alpha_inundacao = 0.5; label_inundacao = 'Área Inundada'
    if not gdf_flood_plot.empty:
        gdf_flood_plot.plot(ax=ax, color=color_inundacao, alpha=alpha_inundacao, zorder=1)
        legend_handles.append(Patch(facecolor=color_inundacao, alpha=alpha_inundacao))
        legend_labels.append(label_inundacao)

    # 2. Canais
    color_canais = 'blue'; linewidth_canais = 0.6; label_canais = 'Rede de Drenagem'
    if gdf_streams_plot is not None and not gdf_streams_plot.empty:
        gdf_streams_plot.plot(ax=ax, color=color_canais, linewidth=linewidth_canais, zorder=2)
        legend_handles.append(Line2D([0], [0], color=color_canais, lw=linewidth_canais))
        legend_labels.append(label_canais)

    # Identificar polígonos OSM não afetados
    gdf_unaffected_plot = gpd.GeoDataFrame(geometry=[])
    if not gdf_polygons_plot.empty:
        unaffected_mask = ~gdf_polygons_plot.index.isin(gdf_affected_plot.index)
        gdf_unaffected_plot = gdf_polygons_plot[unaffected_mask]

    # 3. Plotar polígonos OSM não afetados
    color_nao_afetadas = 'gray'; alpha_nao_afetadas = 0.6; label_nao_afetadas = 'Polígonos OSM Não Afetados'
    if not gdf_unaffected_plot.empty:
        gdf_unaffected_plot.plot(ax=ax, color=color_nao_afetadas, alpha=alpha_nao_afetadas, zorder=3)
        legend_handles.append(Patch(facecolor=color_nao_afetadas, alpha=alpha_nao_afetadas))
        legend_labels.append(label_nao_afetadas)

    # 4. Plotar polígonos OSM afetados
    color_afetadas = 'red'; label_afetadas = 'Polígonos OSM Afetados'
    if not gdf_affected_plot.empty:
        gdf_affected_plot.plot(ax=ax, color=color_afetadas, zorder=4)
        legend_handles.append(Patch(facecolor=color_afetadas))
        legend_labels.append(label_afetadas)
    # ... (restante do código para caso todas sejam afetadas ou nenhuma)

    # ***** AJUSTE AQUI *****
    # 5. Bacia (Topo e Linha mais grossa)
    color_bacia = 'purple'; linewidth_bacia = 1.5; label_bacia = 'Limite da Bacia' # Aumentei linewidth_bacia
    if gdf_catchment_plot is not None and not gdf_catchment_plot.empty:
        gdf_catchment_plot.boundary.plot(ax=ax, color=color_bacia, linewidth=linewidth_bacia, zorder=5) # Mantém zorder=5
        legend_handles.append(Line2D([0], [0], color=color_bacia, lw=linewidth_bacia))
        legend_labels.append(label_bacia)
    # ************************


    # Adicionar anotação com a contagem
    ax.text(0.98, 0.98, f"Polígonos OSM Afetados: {count_affected_polygons}",
            transform=ax.transAxes, fontsize=12,
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7))

    # Configurações finais do mapa e Legenda Personalizada
    ax.set_title('Impacto da Inundação em Polígonos OSM')
    ax.set_xlabel('Coordenada X (Metros)')
    ax.set_ylabel('Coordenada Y (Metros)')

    if legend_handles:
        # Reordenar handles/labels para a ordem desejada na legenda
        desired_order = ['Limite da Bacia', 'Área Inundada', 'Rede de Drenagem', 'Polígonos OSM Não Afetados', 'Polígonos OSM Afetados']
        ordered_handles = []
        ordered_labels = []
        current_labels_dict = dict(zip(legend_labels, legend_handles))
        for label in desired_order:
             if label in current_labels_dict:
                 ordered_handles.append(current_labels_dict[label])
                 ordered_labels.append(label)

        ax.legend(handles=ordered_handles, labels=ordered_labels, title="Legenda")


    plt.tight_layout()
    plt.savefig(output_map_path, dpi=300, bbox_inches='tight')
    print(f"Mapa salvo como '{output_map_path}'")
    plt.show()

    print("Célula 6 concluída com sucesso!")

except Exception as e:
    print(f"Erro inesperado na Célula 6: {e}")
    raise